# 🍽️ Where to DINE - 餐厅数据采集
## 方法2: OpenStreetMap (OSM)

---

### 📋 功能说明
本notebook使用OpenStreetMap数据获取纽约市的餐厅信息

### ⚙️ 准备工作
1. ✅ 无需API Key（完全免费）
2. ✅ 使用OSMnx和Overpass API
3. ✅ 符合OSM使用条款

### 📊 输出数据
- 文件名: `restaurants_nyc_osm.csv`
- 包含: 餐厅名称、坐标、菜系类型、营业时间等

### ✨ 优势
- 完全免费，无配额限制
- 社区维护，数据开放
- 可获取详细的POI属性

---

## 1️⃣ 安装依赖库

In [ ]:
# 安装OSMnx和相关库
!pip install osmnx geopandas pandas numpy matplotlib requests -q

print("✅ 依赖库安装完成！")

## 2️⃣ 导入库

In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import requests
import json
import time

# 配置OSMnx
ox.settings.use_cache = True
ox.settings.log_console = False

print("✅ 库导入成功！")
print(f"OSMnx版本: {ox.__version__}")

## 3️⃣ 配置NYC搜索区域

In [ ]:
# NYC边界坐标
NYC_BOUNDS = {
    'north': 40.9176,
    'south': 40.4774,
    'east': -73.7004,
    'west': -74.2591
}

# 或者使用行政区名称（更准确）
NYC_PLACE_NAME = "New York, New York, USA"

# OSM标签 - 餐厅
RESTAURANT_TAGS = {
    'amenity': 'restaurant'
}

# 也可以获取其他餐饮类型
ALL_FOOD_TAGS = {
    'amenity': ['restaurant', 'cafe', 'fast_food', 'bar', 'pub']
}

print("✅ 配置完成！")
print(f"📍 搜索区域: NYC")
print(f"🏷️  搜索标签: {RESTAURANT_TAGS}")

## 4️⃣ 方法A: 使用OSMnx获取餐厅数据（推荐）

这是最简单直接的方法

In [ ]:
print("🚀 开始从OpenStreetMap获取餐厅数据...")
print("⏳ 这可能需要2-5分钟，请耐心等待...\n")

try:
    # 使用边界框获取餐厅数据
    restaurants_gdf = ox.geometries_from_bbox(
        north=NYC_BOUNDS['north'],
        south=NYC_BOUNDS['south'],
        east=NYC_BOUNDS['east'],
        west=NYC_BOUNDS['west'],
        tags=RESTAURANT_TAGS
    )
    
    print(f"✅ 成功获取 {len(restaurants_gdf)} 个餐厅POI")
    
except Exception as e:
    print(f"❌ 获取数据时出错: {e}")
    print("💡 尝试使用地名搜索...")
    
    # 备用方案：使用地名搜索
    restaurants_gdf = ox.geometries_from_place(
        NYC_PLACE_NAME,
        tags=RESTAURANT_TAGS
    )
    
    print(f"✅ 成功获取 {len(restaurants_gdf)} 个餐厅POI")

## 5️⃣ 检查原始数据

In [ ]:
print("📊 原始数据信息:")
print(f"  总记录数: {len(restaurants_gdf)}")
print(f"  列数: {len(restaurants_gdf.columns)}")
print(f"\n数据类型:")
print(restaurants_gdf.geom_type.value_counts())

print(f"\n可用字段（前20个）:")
print(list(restaurants_gdf.columns[:20]))

print(f"\n数据预览:")
display(restaurants_gdf.head(3))

## 6️⃣ 处理和清洗数据

In [ ]:
print("🔧 开始处理数据...\n")

# 复制数据避免修改原始数据
df = restaurants_gdf.copy()

# 1. 提取中心点坐标（对于polygon/multipolygon类型）
def get_centroid_coords(geometry):
    """提取几何中心点的坐标"""
    try:
        centroid = geometry.centroid
        return centroid.y, centroid.x  # lat, lon
    except:
        return None, None

df['latitude'], df['longitude'] = zip(*df['geometry'].apply(get_centroid_coords))

# 2. 删除没有坐标的记录
df = df.dropna(subset=['latitude', 'longitude'])
print(f"✅ 删除无效坐标后剩余: {len(df)} 条记录")

# 3. 选择有用的列（如果存在）
columns_to_keep = ['name', 'latitude', 'longitude']
optional_columns = ['cuisine', 'phone', 'website', 'opening_hours', 
                   'addr:street', 'addr:housenumber', 'addr:city',
                   'amenity', 'brand', 'operator']

for col in optional_columns:
    if col in df.columns:
        columns_to_keep.append(col)

# 创建最终数据框
df_final = df[columns_to_keep].copy()

# 4. 添加类别列
df_final['category'] = 'restaurant'

# 5. 重命名列（使其更易读）
rename_dict = {
    'addr:street': 'street',
    'addr:housenumber': 'house_number',
    'addr:city': 'city'
}
df_final = df_final.rename(columns=rename_dict)

# 6. 处理菜系信息（可能有多个值）
if 'cuisine' in df_final.columns:
    df_final['cuisine'] = df_final['cuisine'].apply(
        lambda x: x if pd.isna(x) else str(x).replace(';', ', ')
    )

print(f"\n✅ 数据处理完成！")
print(f"最终数据集包含 {len(df_final)} 个餐厅")
print(f"\n最终列名: {list(df_final.columns)}")

## 7️⃣ 数据统计分析

In [ ]:
print("📊 数据统计分析\n")

print(f"总餐厅数: {len(df_final)}")

# 有名称的餐厅
named_count = df_final['name'].notna().sum()
print(f"有名称的餐厅: {named_count} ({named_count/len(df_final)*100:.1f}%)")

# 有菜系信息的餐厅
if 'cuisine' in df_final.columns:
    cuisine_count = df_final['cuisine'].notna().sum()
    print(f"有菜系信息的餐厅: {cuisine_count} ({cuisine_count/len(df_final)*100:.1f}%)")
    
    # 最常见的菜系
    print(f"\n🍴 最常见的菜系类型（Top 10）:")
    cuisine_types = df_final['cuisine'].dropna().str.split(', ').explode()
    top_cuisines = cuisine_types.value_counts().head(10)
    for i, (cuisine, count) in enumerate(top_cuisines.items(), 1):
        print(f"  {i}. {cuisine}: {count}")

# 地理分布
print(f"\n📍 地理范围:")
print(f"  纬度: {df_final['latitude'].min():.4f} ~ {df_final['latitude'].max():.4f}")
print(f"  经度: {df_final['longitude'].min():.4f} ~ {df_final['longitude'].max():.4f}")

## 8️⃣ 数据可视化

In [ ]:
# 创建地图可视化
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 图1: 餐厅位置密度图
axes[0].scatter(
    df_final['longitude'], 
    df_final['latitude'], 
    alpha=0.5, 
    s=5,
    c='red',
    edgecolors='none'
)
axes[0].set_xlabel('经度', fontsize=12)
axes[0].set_ylabel('纬度', fontsize=12)
axes[0].set_title(f'NYC餐厅位置分布\n(OpenStreetMap, n={len(df_final)})', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# 图2: 热力图（密度）
axes[1].hexbin(
    df_final['longitude'],
    df_final['latitude'],
    gridsize=30,
    cmap='YlOrRd',
    mincnt=1
)
axes[1].set_xlabel('经度', fontsize=12)
axes[1].set_ylabel('纬度', fontsize=12)
axes[1].set_title('餐厅密度热力图', fontsize=14, fontweight='bold')
plt.colorbar(axes[1].collections[0], ax=axes[1], label='餐厅数量')

plt.tight_layout()
plt.show()

print("✅ 可视化完成！")

## 9️⃣ 菜系类型分布可视化（如果有数据）

In [ ]:
if 'cuisine' in df_final.columns and df_final['cuisine'].notna().sum() > 0:
    # 提取所有菜系类型
    cuisine_types = df_final['cuisine'].dropna().str.split(', ').explode()
    top_15_cuisines = cuisine_types.value_counts().head(15)
    
    # 创建柱状图
    fig, ax = plt.subplots(figsize=(12, 6))
    top_15_cuisines.plot(kind='barh', ax=ax, color='steelblue')
    ax.set_xlabel('餐厅数量', fontsize=12)
    ax.set_ylabel('菜系类型', fontsize=12)
    ax.set_title('NYC最常见的15种菜系', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # 添加数值标签
    for i, v in enumerate(top_15_cuisines.values):
        ax.text(v + 5, i, str(v), va='center')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ 菜系分布图生成完成！")
else:
    print("ℹ️  没有足够的菜系数据用于可视化")

## 🔟 保存数据到CSV

In [ ]:
# 保存到CSV
output_filename = 'restaurants_nyc_osm.csv'
df_final.to_csv(output_filename, index=False, encoding='utf-8')

print(f"✅ 数据已保存到: {output_filename}")

# 显示文件信息
import os
file_size = os.path.getsize(output_filename)
print(f"📁 文件大小: {file_size / 1024:.2f} KB")
print(f"📊 总记录数: {len(df_final)}")

# 如果在Colab中，提供下载
try:
    from google.colab import files
    files.download(output_filename)
    print("⬇️ 文件下载已开始...")
except:
    print("💡 如需下载，请在Colab左侧文件栏找到文件右键下载")

## 1️⃣1️⃣ 保存为GeoJSON格式（可选）

GeoJSON格式更适合在地图上显示

In [ ]:
from shapely.geometry import Point

# 创建GeoDataFrame
geometry = [Point(xy) for xy in zip(df_final['longitude'], df_final['latitude'])]
gdf_output = gpd.GeoDataFrame(df_final, geometry=geometry, crs='EPSG:4326')

# 保存为GeoJSON
geojson_filename = 'restaurants_nyc_osm.geojson'
gdf_output.to_file(geojson_filename, driver='GeoJSON')

print(f"✅ GeoJSON文件已保存: {geojson_filename}")
print(f"📁 文件大小: {os.path.getsize(geojson_filename) / 1024:.2f} KB")

# 尝试下载
try:
    from google.colab import files
    files.download(geojson_filename)
    print("⬇️ GeoJSON文件下载已开始...")
except:
    pass

## 1️⃣2️⃣ 备用方法B: 使用Overpass API（高级用户）

如果OSMnx方法遇到问题，可以直接使用Overpass API

In [ ]:
def query_overpass_api(bbox, tags='amenity=restaurant'):
    """
    直接查询Overpass API
    
    参数:
        bbox: (south, west, north, east)
        tags: OSM标签查询字符串
    """
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    # 构建Overpass查询语句
    overpass_query = f"""
    [out:json][timeout:90];
    (
      node["{tags}"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["{tags}"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      relation["{tags}"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
    );
    out center;
    """
    
    print("🔍 正在查询Overpass API...")
    response = requests.get(overpass_url, params={'data': overpass_query})
    
    if response.status_code == 200:
        data = response.json()
        return data['elements']
    else:
        print(f"❌ 查询失败: {response.status_code}")
        return []

# 示例使用（仅在需要时取消注释运行）
# bbox = (NYC_BOUNDS['south'], NYC_BOUNDS['west'], NYC_BOUNDS['north'], NYC_BOUNDS['east'])
# results = query_overpass_api(bbox)
# print(f"获取到 {len(results)} 个结果")

print("💡 Overpass API方法已定义（备用）")

## 1️⃣3️⃣ 数据预览和示例

In [ ]:
print("📋 数据示例（前10条记录）:\n")
display(df_final.head(10))

print("\n📊 数据类型:")
print(df_final.dtypes)

print("\n🔍 数据完整性检查:")
print(df_final.isnull().sum())

print("\n✅ 数据质量良好，可以用于Where to DINE项目！")

## 📊 数据采集总结报告

In [ ]:
print("=" * 70)
print("📊 OpenStreetMap餐厅数据采集总结")
print("=" * 70)

print(f"\n📅 采集时间: {datetime.now().strftime('%Y年%m月%d日 %H:%M:%S')}")
print(f"🗺️  数据来源: OpenStreetMap (OSM)")
print(f"📍 搜索区域: 纽约市 (NYC)")

print(f"\n📊 数据统计:")
print(f"  ├─ 总餐厅数量: {len(df_final)}")
print(f"  ├─ 有名称的餐厅: {df_final['name'].notna().sum()} ({df_final['name'].notna().sum()/len(df_final)*100:.1f}%)")

if 'cuisine' in df_final.columns:
    print(f"  ├─ 有菜系信息: {df_final['cuisine'].notna().sum()} ({df_final['cuisine'].notna().sum()/len(df_final)*100:.1f}%)")

print(f"  └─ 数据完整性: ✅ 良好")

print(f"\n🌍 地理覆盖:")
print(f"  ├─ 纬度范围: {df_final['latitude'].min():.4f}° ~ {df_final['latitude'].max():.4f}°")
print(f"  └─ 经度范围: {df_final['longitude'].min():.4f}° ~ {df_final['longitude'].max():.4f}°")

print(f"\n💾 输出文件:")
print(f"  ├─ CSV格式: {output_filename}")
print(f"  └─ GeoJSON格式: {geojson_filename}")

print(f"\n✨ 优势:")
print(f"  ├─ ✅ 完全免费，无API配额限制")
print(f"  ├─ ✅ 社区维护，数据开放")
print(f"  ├─ ✅ 包含丰富的属性信息")
print(f"  └─ ✅ 符合Where to DINE项目需求")

print("\n" + "=" * 70)
print("🎉 数据采集完成！可以直接用于项目分析")
print("=" * 70)

print("\n📝 下一步:")
print("  1. 下载生成的CSV文件")
print("  2. 上传到项目的 data/raw/ 目录")
print("  3. 运行聚类脚本: 01_cluster_restaurants.py")
print("  4. 继续处理出租车数据")

## ⚖️ 数据使用声明

### OpenStreetMap数据许可

本notebook使用的数据来自OpenStreetMap，受Open Database License (ODbL)保护。

**使用要求:**
- ✅ 必须注明数据来源: © OpenStreetMap contributors
- ✅ 可以自由使用、修改和分发
- ✅ 衍生作品也应开放

**致谢:**
- 感谢OpenStreetMap社区贡献者
- 感谢OSMnx开发团队

**详细信息:**
- OSM版权: https://www.openstreetmap.org/copyright
- ODbL许可: https://opendatacommons.org/licenses/odbl/

---

**在你的项目中使用时，请添加以下声明:**

```
数据来源: © OpenStreetMap contributors
许可协议: Open Database License (ODbL)
```